<a href="https://colab.research.google.com/github/beyzaru/beyza/blob/main/beyza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from sklearn.metrics import precision_score, f1_score

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score

In [ ]:
os.chdir("/content/drive/MyDrive")

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Adsız klasör/titanik.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.shape

(891, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df.keys

<bound method NDFrame.keys of      PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0              1         0       3  ...   7.2500   NaN         S
1              2         1       1  ...  71.2833   C85         C
2              3         1       3  ...   7.9250   NaN         S
3              4         1       1  ...  53.1000  C123         S
4              5         0       3  ...   8.0500   NaN         S
..           ...       ...     ...  ...      ...   ...       ...
886          887         0       2  ...  13.0000   NaN         S
887          888         1       1  ...  30.0000   B42         S
888          889         0       3  ...  23.4500   NaN         S
889          890         1       1  ...  30.0000  C148         C
890          891         0       3  ...   7.7500   NaN         Q

[891 rows x 12 columns]>

In [ ]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
df.groupby("Sex").agg({"Age":"median"})

,Age
Sex,
female,27.0
male,29.0


In [ ]:
df.loc[(df.Sex == "female")& (pd.isna(df.Age)), "Age"] = 27.0
df.loc[(df.Sex == "male")& (pd.isna(df.Age)), "Age"] = 29.0

In [ ]:
df = df.fillna ("U")

In [ ]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
df = df.drop (columns = ["Ticket", "PassengerId", "Name", "Cabin"])

In [ ]:
df[["Sex", "Embarked"]] = df [["Sex", "Embarked"]].astype("category")

In [ ]:
cat_columns = df.select_dtypes(["category"]).columns
df[cat_columns]= df[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
df.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
5,0,3,1,29.0,0,0,8.4583,1
6,0,1,1,54.0,0,0,51.8625,2
7,0,3,1,2.0,3,1,21.0750,2
8,1,3,0,27.0,0,2,11.1333,2
9,1,2,0,14.0,1,0,30.0708,0


In [ ]:
df.groupby("Survived").size()

Survived
0    549
1    342
dtype: int64

In [ ]:
df["Age"].isna().sum() #kontrol amaçlı

0

In [ ]:
df.dtypes

Survived      int64
Pclass        int64
Sex            int8
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked       int8
dtype: object

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(df.loc[:,"Pclass" : "Embarked"],
                                                   df["Survived"], test_size=0.25)

In [ ]:
knn= KNeighborsClassifier()
knn= knn.fit(X_train, Y_train)
knn_pred=knn.predict(X_test)

In [ ]:
nb= GaussianNB()
nb = nb.fit(X_train, Y_train)
nb_pred=nb.predict(X_test)

In [ ]:
dtc=KNeighborsClassifier()
dtc=dtc.fit(X_train, Y_train)
dtc_pred=dtc.predict(X_test)

In [ ]:
lreg=LogisticRegression()
lreg=lreg.fit(X_train,Y_train)
lreg_pred=lreg.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
confusion_matrix(Y_test, knn_pred)

array([[107,  25],
       [ 36,  55]])

In [ ]:
recall_score(Y_test, knn_pred)

0.6043956043956044

In [ ]:
55/(55+33)

0.625

In [ ]:
print(classification_report(Y_test, knn_pred))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       132
           1       0.69      0.60      0.64        91

    accuracy                           0.73       223
   macro avg       0.72      0.71      0.71       223
weighted avg       0.72      0.73      0.72       223



In [ ]:
confusion_matrix(Y_test,nb_pred)

array([[113,  19],
       [ 26,  65]])

In [ ]:
recall_score(Y_test,nb_pred)

0.7142857142857143

In [ ]:
print(classification_report(Y_test,nb_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       132
           1       0.77      0.71      0.74        91

    accuracy                           0.80       223
   macro avg       0.79      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223



In [ ]:
confusion_matrix(Y_test, dtc_pred)

array([[107,  25],
       [ 36,  55]])

In [ ]:
recall_score(Y_test, dtc_pred)

0.6043956043956044

In [ ]:
print(classification_report(Y_test,dtc_pred))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       132
           1       0.69      0.60      0.64        91

    accuracy                           0.73       223
   macro avg       0.72      0.71      0.71       223
weighted avg       0.72      0.73      0.72       223



In [ ]:
confusion_matrix(Y_test, lreg_pred)

array([[116,  16],
       [ 29,  62]])

In [ ]:
recall_score(Y_test, lreg_pred)

0.6813186813186813

In [ ]:
print(classification_report(Y_test,lreg_pred))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       132
           1       0.79      0.68      0.73        91

    accuracy                           0.80       223
   macro avg       0.80      0.78      0.79       223
weighted avg       0.80      0.80      0.80       223



In [ ]:
precision_recall_fscore_support(Y_test,nb_pred)

(array([0.81294964, 0.77380952]),
 array([0.85606061, 0.71428571]),
 array([0.83394834, 0.74285714]),
 array([132,  91]))

In [ ]:
precision_score(Y_test, nb_pred)

0.7738095238095238

In [ ]:
f1_score(Y_test, nb_pred)

0.7428571428571429

In [ ]:
roc_auc_score(Y_test,nb_pred)

0.7851731601731603

In [ ]:
roc_auc_score(Y_test, lreg_pred)

0.7800532800532801

In [ ]:
model = XGBClassifier(n_estimators = 100)
results = cross_val_score(model, df.loc[:,'Pclass' : 'Embarked'], df['Survived'], scoring='roc_auc', cv=3)
print(results.mean())

0.8453887450867605


In [ ]:
estimators = []
estimators.append(("lr", LogisticRegression()))
estimators.append(("dt", DecisionTreeClassifier()))
estimators.append(("nb", GaussianNB()))

In [ ]:
ensemble=VotingClassifier(estimators)
ensemble.fit(X_train, Y_train)
ensemble_pred=ensemble.predict (X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
precision_score(Y_test,ensemble_pred)

0.8076923076923077

In [ ]:
knn=KNeighborsClassifier()
lr=LogisticRegression()
dtc=DecisionTreeClassifier()
nb=GaussianNB()

In [ ]:
models=[knn,lr,dtc,nb]
meta_alg=XGBClassifier(n_estimators=40)

In [ ]:
meta_mtrx=np.empty((df.shape[0], len(models)))

In [ ]:
for n, model in enumerate(models):
  meta_mtrx[:,n]=cross_val_predict(model, df.loc[:,"Pclass" : "Embarked"],
                                   df["Survived"], cv=5, method="predict")
  base_algorithm=model.fit(X_train, Y_train)
  predicted=model.predict(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
meta_model=meta_alg.fit(meta_mtrx,df["Survived"])
meta_mtrx_test=np.empty((df.shape[0],len(models)))

In [ ]:
for n, model in enumerate(models):
  meta_mtrx_test[:,n] = model.predict(df.loc[:,"Pclass": "Embarked"])
  meta_predict=meta_alg.predict(meta_mtrx_test)

In [ ]:
precision_score(df["Survived"], meta_predict)

0.8481012658227848

In [ ]:
train,valid,train_true,valid_true=train_test_split(X_train,Y_train,
                                                   train_size=0.5,
                                                   random_state=17)

In [ ]:
knn=KNeighborsClassifier().fit(train,train_true)

In [ ]:
lr=LogisticRegression().fit(train,train_true)

In [ ]:
dtc= DecisionTreeClassifier().fit(train,train_true)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc= SVC().fit(train, train_true)

In [ ]:
models=[knn,lr,dtc,svc]

In [ ]:
from xgboost.sklearn import XGBRFClassifier
meta=XGBRFClassifier(n_estimators=40)

In [ ]:
meta_mtrx=np.empty((valid.shape[0], len(models)))

In [ ]:
from sklearn.metrics import accuracy_score
for n, model in enumerate (models):
  meta_mtrx[:,n]=model.predict(valid)
  predicted=model.predict(X_test)
  print(accuracy_score(Y_test,predicted))

0.695067264573991
0.7982062780269058
0.7713004484304933
0.6636771300448431


In [ ]:
meta_model=meta.fit(meta_mtrx,valid_true)

In [ ]:
meta_mtrx_test= np.empty((X_test.shape[0], len(models)))

In [ ]:
for n, model in enumerate (models):
  meta_mtrx_test[:,n]=model.predict(X_test)

In [ ]:
meta_predict=meta.predict(meta_mtrx_test)

In [ ]:
print(accuracy_score(Y_test,meta_predict))

0.7713004484304933
